In [7]:
# Setting the Video Capture configuration
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [8]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Reading image frames
while(cap.isOpened()):
    ret, frame = cap.read()
    roi = frame[0:500, 0:500]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    # Applying adaptive thresholding
    gray_blur = cv2.GaussianBlur(gray, (15, 15), 0)
    #상황에 맞는 Threshold 사용
    thresh = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 3, 1)
    #ret, thresh = cv2.threshold(gray_blur, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Morphology
    kernel = np.ones((3, 3), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations = 2)
    
    # Contour detection and filtering
    cont_img = closing.copy()
    contours, hierarchy = cv2.findContours(cont_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 700 or area > 25000:
            continue
        
        if len(cnt) < 5:
            continue
            
        ellipse = cv2.fitEllipse(cnt)
        cv2.ellipse(roi, ellipse, (0, 255, 0), 3)
        
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        imgcontour = cv2.drawContours(roi, [box], 0, (0, 0, 255), 2)
    
    cv2.imshow('result', roi)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1